# Insert Vectors into S3 Vector Index

This notebook demonstrates how to insert predefined text into the S3 Vector Index by:
1. Loading predefined text chunks.
2. Generating embeddings for each chunk using the Titan v2 model.
3. Creating metadata with unique IDs (similar to nanoid in JS) and raw text.
4. Inserting vectors into the S3 Vector Index.

In [ ]:
# Import necessary libraries
import boto3
import os
import json
import secrets
import string
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

print("✅ Libraries imported and environment variables loaded.")

In [ ]:
def generate_nanoid(size=21):
    """
    Generate a unique ID similar to nanoid in JavaScript.
    Uses URL-safe characters (A-Za-z0-9_-)
    """
    alphabet = string.ascii_letters + string.digits + '_-'
    return ''.join(secrets.choice(alphabet) for _ in range(size))

# Test the function
print(f"Sample ID: {generate_nanoid()}")

In [ ]:
# Configuration
aws_region = os.getenv("AWS_REGION", "us-east-1")
aws_profile = os.getenv("AWS_PROFILE", "default")
bedrock_embedding_model_id = os.getenv("BEDROCK_EMBEDDING_MODEL_ID", "amazon.titan-embed-text-v2:0")
s3_vector_bucket_name = os.getenv("S3_VECTOR_BUCKET_NAME")
s3_vector_index_name = os.getenv("S3_VECTOR_INDEX_NAME")

print(f"AWS Region: {aws_region}")
print(f"AWS Profile: {aws_profile}")
print(f"Bedrock Embedding Model: {bedrock_embedding_model_id}")
print(f"S3 Vector Bucket: {s3_vector_bucket_name}")
print(f"S3 Vector Index: {s3_vector_index_name}")

# Setup AWS Session
session = boto3.Session(profile_name=aws_profile, region_name=aws_region)
bedrock_client = session.client("bedrock-runtime")

# Initialize S3 Vectors Client
try:
    s3_vectors_client = session.client("s3vectors")
    print("✅ S3 Vectors client initialized.")
except Exception as e:
    print(f"⚠️ Failed to initialize 's3vectors' client. Ensure your boto3 version supports it. Error: {e}")
    s3_vectors_client = None

## Step 1: Define Predefined Text Chunks
Define the text chunks that will be inserted into the vector index.

In [ ]:
# Predefined text chunks to insert
predefined_texts = [
    "Dynamic dispatch: In traditional distributed systems, the dynamic dispatch pattern selects and invokes specific services at runtime based on incoming event attributes, such as event type, source, and payload. This is commonly implemented using Amazon EventBridge, which can evaluate and route incoming events to appropriate targets (for example, AWS Lambda functions AWS Step Functions, or Amazon Elastic Container Service tasks).",
    
    "An application emits an event (for example, {\"type\": \"orderCreated\", \"priority\": \"high\"}). Amazon EventBridge evaluates the event against its routing rules. Based on an event's attributes, the system dynamically dispatches to the following: HighPriorityOrderProcessor (service A), StandardOrderProcessor (service B), UpdateOrderProcessor (service C). This pattern supports loose coupling, domain-based specialization, and runtime extensibility. This allows systems to respond intelligently to changing requirements and event semantics.",
    
    "LLM-based routing: In agentic systems, routing also performs dynamic task delegation - but instead of Amazon EventBridge rules or metadata filters, the LLM classifies and interprets the user's intent through natural language. The result is a flexible, semantic, and adaptive form of dispatching.",
    
    "Agent router: This architecture enables rich intent-based dispatching without predefined schemas or event types, which is ideal for unstructured input and complex queries. A user submits the request 'Can you help me review my contract terms?' The LLM interprets this as a legal document task. The agent routes the task to one or more of the following: Contract review prompt template, Legal reasoning subagent, Document parsing tool.",
    
    "Agent router workflow: A user submits a natural language request through an SDK. An Amazon Bedrock agent uses an LLM to classify the task (for example, legal, technical, or scheduling). The agent dynamically routes the task through an action group to invoke the required agent: Domain-specific agent, Specialized tool chain, Custom prompt configuration. The selected handler processes the task and returns a tailored response.",
    
    "Takeaways: Where traditional dynamic dispatch uses Amazon EventBridge rules for routing based on structured event attributes, agentic routing uses LLMs to semantically classify and route tasks based on meaning and intent. This expands the system's flexibility by enabling: Broader input understanding, Intelligent fallback and tool selection, Natural extensibility through new agent roles or prompt styles. Agentic routing replaces rigid rules with dynamic cognitive dispatching, which allows systems to evolve with language rather than code.",
    
    "Parallelization and scatter-gather patterns: Many advanced reasoning and generation tasks - such as summarizing large documents, evaluating multiple solution paths, or comparing diverse perspectives - benefit from the parallel execution of prompts. Traditional sequential workflows fall short when scalability, responsiveness, and fault tolerance are required. To overcome this, LLM-based parallelization can be reimagined using an event-driven scatter-gather pattern, where tasks are dynamically fanned out to autonomous agents and results intelligently synthesized.",
    
    "Scatter-gather: In distributed systems, a scatter-gather pattern sends tasks to multiple services or processing units in parallel, waits for their responses, and then aggregates results into a consolidated output. Unlike fan-out, scatter-gather is coordinated because it expects responses and usually applies logic to combine, compare, and select results.",
    
    "Common implementations for parallelization and scatter-gather include the following: AWS Step Functions map a state for parallel task execution, AWS Lambda with concurrency, coordinating results from multiple invoked functions, Amazon EventBridge with correlation IDs and aggregation workflows, Custom controller pattern to manage fan-out and gather results by using Amazon Simple Storage Service (Amazon S3), Amazon DynamoDB, or queues."
]

print(f"✅ Defined {len(predefined_texts)} text chunks for insertion.")
for i, text in enumerate(predefined_texts, 1):
    print(f"{i}. {text[:80]}...")

## Step 2: Generate Embeddings
Use the Titan v2 model to generate embeddings for each text chunk.

In [ ]:
def get_embedding(text, model_id=bedrock_embedding_model_id):
    """
    Generate embedding for a given text using Amazon Bedrock Titan model.
    """
    try:
        body = json.dumps({
            "inputText": text,
            # Optional: "dimensions": 1024, "normalize": True
        })
        
        response = bedrock_client.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )
        
        response_body = json.loads(response.get("body").read())
        embedding = response_body.get("embedding")
        return embedding
    except Exception as e:
        print(f"❌ Error generating embedding: {e}")
        return None

# Test embedding generation
test_embedding = get_embedding(predefined_texts[0])
if test_embedding:
    print(f"✅ Successfully generated embedding with dimension: {len(test_embedding)}")

## Step 3: Insert Vectors into S3 Vector Index
For each text chunk:
1. Generate a unique ID (similar to nanoid)
2. Create embedding
3. Create metadata with id and chunk
4. Insert into the vector index

In [ ]:
def insert_vector(text, bucket_name, index_name):
    """
    Insert a single text chunk into the S3 Vector Index.
    """
    if not s3_vectors_client:
        print("❌ S3 Vectors client is not initialized.")
        return False
    
    if not bucket_name or not index_name:
        print("⚠️ S3_VECTOR_BUCKET_NAME or S3_VECTOR_INDEX_NAME is not set.")
        return False
    
    # Generate unique ID
    unique_id = generate_nanoid()
    
    # Generate embedding
    embedding = get_embedding(text)
    if not embedding:
        print(f"❌ Failed to generate embedding for text: {text[:50]}...")
        return False
    
    # Create metadata
    metadata = {
        "id": unique_id,
        "chunk": text
    }
    
    try:
        # Insert vector into S3 Vector Index
        response = s3_vectors_client.put_vectors(
            vectorBucketName=bucket_name,
            indexName=index_name,
            vectors=[
                {
                    "key": unique_id,
                    "data": {"float32": embedding},
                    "metadata": metadata
                }
            ]
        )
        
        print(f"✅ Inserted vector with ID: {unique_id}")
        print(f"   Text preview: {text[:80]}...")
        return True
        
    except Exception as e:
        print(f"❌ Error inserting vector: {e}")
        return False

# Insert all predefined texts
print("\n🚀 Starting vector insertion...\n")
success_count = 0
fail_count = 0

for i, text in enumerate(predefined_texts, 1):
    print(f"\n[{i}/{len(predefined_texts)}] Processing...")
    if insert_vector(text, s3_vector_bucket_name, s3_vector_index_name):
        success_count += 1
    else:
        fail_count += 1

print(f"\n\n📊 Insertion Summary:")
print(f"   ✅ Successful: {success_count}")
print(f"   ❌ Failed: {fail_count}")
print(f"   📝 Total: {len(predefined_texts)}")

## Step 4: Verify Insertion (Optional)
Perform a test query to verify that the vectors were inserted successfully.

In [13]:
# Test query to verify insertion
test_query = "Explain how EventBridge works in LLM Workflow context?"
print(f"🔍 Testing with query: '{test_query}'\n")

# Generate embedding for test query
query_embedding = get_embedding(test_query)

if query_embedding and s3_vectors_client:
    try:
        response = s3_vectors_client.query_vectors(
            vectorBucketName=s3_vector_bucket_name,
            indexName=s3_vector_index_name,
            queryVector={"float32": query_embedding},
            topK=3,
            returnDistance=True,
            returnMetadata=True
        )
        
        vectors = response.get('vectors', [])
        print(f"✅ Found {len(vectors)} similar vectors:\n")
        
        for i, result in enumerate(vectors, 1):
            metadata = result.get('metadata', {})
            chunk_text = metadata.get('chunk', '')
            vector_id = metadata.get('id', '')
            distance = result.get('distance', 0)
            
            print(f"{i}. ID: {vector_id}")
            print(f"   Distance: {distance:.4f}")
            print(f"   Text: {chunk_text}")
            print()
            
    except Exception as e:
        print(f"❌ Error querying vectors: {e}")
else:
    print("⚠️ Skipping verification - client not initialized or embedding failed.")

🔍 Testing with query: 'Explain how EventBridge works in LLM Workflow context?'

✅ Found 3 similar vectors:

1. ID: HlgHN84M1CUmWH3q5tzHl
   Distance: 0.4293
   Text: An application emits an event (for example, {"type": "orderCreated", "priority": "high"}). Amazon EventBridge evaluates the event against its routing rules. Based on an event's attributes, the system dynamically dispatches to the following: HighPriorityOrderProcessor (service A), StandardOrderProcessor (service B), UpdateOrderProcessor (service C). This pattern supports loose coupling, domain-based specialization, and runtime extensibility. This allows systems to respond intelligently to changing requirements and event semantics.

2. ID: Pv2Y1r0zYKxdFRrToZ0fo
   Distance: 0.5672
   Text: LLM-based routing: In agentic systems, routing also performs dynamic task delegation - but instead of Amazon EventBridge rules or metadata filters, the LLM classifies and interprets the user's intent through natural language. The result is